In [23]:
'''
모델 매개변수 최적화하기

이제 모델과 데이터가 준비되었으니, 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례입니다. 모델을 학습하는 과정은 반복적인 과정을 거칩니다.
각 반복단계에서 모델은 출력을 추측하고, 추측과 정답사이의 loss를 계산하고, 매개변수에 대한 오류의 도함수(derivative)를 수집ㅈ한 뒤, 경사하강법을 사용하여
이 parameter들을 최적화(optimize)합니다.
'''

'\n모델 매개변수 최적화하기\n\n이제 모델과 데이터가 준비되었으니, 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례입니다. 모델을 학습하는 과정은 반복적인 과정을 거칩니다.\n각 반복단계에서 모델은 출력을 추측하고, 추측과 정답사이의 loss를 계산하고, 매개변수에 대한 오류의 도함수(derivative)를 수집ㅈ한 뒤, 경사하강법을 사용하여\n이 parameter들을 최적화(optimize)합니다.\n'

In [24]:
'''
기본(pre-requisite)코드
'''

'\n기본(pre-requisite)코드\n'

In [67]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() #https://dojang.io/mod/page/view.php?id=2386
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x=self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [68]:
'''
하이퍼파라미터(Hyperparameter)

하이퍼파라미터(Hyperparameter)는 모델 최적화 과정을 제어할 수 잇는 조절 가능한 매개변수입니다. 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)
에 영향을 미칠 수 있습니다.

학습시에는 다음과 같은 하ㅣ퍼파라미터를 정의합니다.
    1. 에폭(epoch)수 - 데이터셋을 반복하는 횟수
    2. 배치크기(batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
    3. 학습률(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율, 값이 작을수혹 학습 속도가 느려지고,
                              값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.
'''

'\n하이퍼파라미터(Hyperparameter)\n\n하이퍼파라미터(Hyperparameter)는 모델 최적화 과정을 제어할 수 잇는 조절 가능한 매개변수입니다. 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)\n에 영향을 미칠 수 있습니다.\n\n학습시에는 다음과 같은 하ㅣ퍼파라미터를 정의합니다.\n    1. 에폭(epoch)수 - 데이터셋을 반복하는 횟수\n    2. 배치크기(batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수\n    3. 학습률(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율, 값이 작을수혹 학습 속도가 느려지고,\n                              값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.\n'

In [69]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [70]:
'''
최적화 단계(Optimization Loop)

하이퍼마라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있습니다. 최적화 단계의 각 반복을 에폭이라고 부릅니다.
하나의 에폭은 다음 두 부분으로 구성됩니다.
    1. 학습단계(train loop)-학습용 데이터셋을 반복하고 최적의 매개변수로 수렴합니다.
    2. 검증/테스트단계(validation/test loop)-모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복합니다.
학습단계(training loop)에서 이러나는 몇 가지 개념들을 간략히 살펴보겠습니다. 최적화 단계(optimization loop)를 보면서 전체 구현 부분으로 건너뛰면 됨.

손실함수(loss function / cost function)
학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제고하지 않을 확률이 높습니다. loss function은 획득한 결과와 실제 값 사이의
틀린 정도(degree fo dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 합니다. 주어진 데이터 샘플을 입력으로 계산한 예측고 정답(label)을 비교하여
손실(loss)을 계산합니다.

일반적으로 손실함수에는 회귀 문제(regression task)에 사용하는 nn.MSELoss(평균 제곱 오차(MSE; Mean Square Error))나 classcification에 사용하는 nn.NLLLoss,
그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss등이 있습니다.

모델의 출력 로짓(logit)을 nn.CrossEntropyLoss에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산합니다.
'''

'\n최적화 단계(Optimization Loop)\n\n하이퍼마라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있습니다. 최적화 단계의 각 반복을 에폭이라고 부릅니다.\n하나의 에폭은 다음 두 부분으로 구성됩니다.\n    1. 학습단계(train loop)-학습용 데이터셋을 반복하고 최적의 매개변수로 수렴합니다.\n    2. 검증/테스트단계(validation/test loop)-모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복합니다.\n학습단계(training loop)에서 이러나는 몇 가지 개념들을 간략히 살펴보겠습니다. 최적화 단계(optimization loop)를 보면서 전체 구현 부분으로 건너뛰면 됨.\n\n손실함수(loss function / cost function)\n학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제고하지 않을 확률이 높습니다. loss function은 획득한 결과와 실제 값 사이의\n틀린 정도(degree fo dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 합니다. 주어진 데이터 샘플을 입력으로 계산한 예측고 정답(label)을 비교하여\n손실(loss)을 계산합니다.\n\n일반적으로 손실함수에는 회귀 문제(regression task)에 사용하는 nn.MSELoss(평균 제곱 오차(MSE; Mean Square Error))나 classcification에 사용하는 nn.NLLLoss,\n그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss등이 있습니다.\n\n모델의 출력 로짓(logit)을 nn.CrossEntropyLoss에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산합니다.\n'

In [71]:
loss_fn = nn.CrossEntropyLoss()

In [72]:
'''
옵티마이저(Optimizer)

최적화는 각 학습 단계에서 모델의 오류를 중이기 위해 모델 매개변수를 조정하는 과정입니다. 최적화 알고리즘은 이 과정이 수행되는 방식
(SGD; Stochastic Gradient Descent)을 정의합니다. 모든 최적화 절차(logic)는 optimizer객체에 캡슐화(encapsulate)됩니다.여기에서는 SGD옵티마이저를 사용하고 있으며
pytorch에는 ADAM이나 RMSProp와 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있습니다.

학습하려는 모델의 매개변수와 learning rate Hyperparameter를 등록하여 옵티마이저를 초기화합니다.
'''

'\n옵티마이저(Optimizer)\n\n최적화는 각 학습 단계에서 모델의 오류를 중이기 위해 모델 매개변수를 조정하는 과정입니다. 최적화 알고리즘은 이 과정이 수행되는 방식\n(SGD; Stochastic Gradient Descent)을 정의합니다. 모든 최적화 절차(logic)는 optimizer객체에 캡슐화(encapsulate)됩니다.여기에서는 SGD옵티마이저를 사용하고 있으며\npytorch에는 ADAM이나 RMSProp와 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있습니다.\n\n학습하려는 모델의 매개변수와 learning rate Hyperparameter를 등록하여 옵티마이저를 초기화합니다.\n'

In [73]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [74]:
'''
학습단계(loop)에서 최적화는 세단계로 이뤄집니다.
    1. optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up)때문에 중복 계산을 막기 위해 반복할 때마다
       명시적으로 0으로 설정합니다.
    2. loss.backwards()를 호출하여 예측손실(prediction loss)을 역전파합니다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.
    3. 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.
'''

'\n학습단계(loop)에서 최적화는 세단계로 이뤄집니다.\n    1. optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up)때문에 중복 계산을 막기 위해 반복할 때마다\n       명시적으로 0으로 설정합니다.\n    2. loss.backwards()를 호출하여 예측손실(prediction loss)을 역전파합니다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.\n    3. 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.\n'

In [75]:
'''
전체 구현

최적화 코드를 반복하여 수행하는 train_loop와 테스트 데이터로 모델의 성늘을 측정하는 test_loop를 정의하였습니다.
'''

'\n전체 구현\n\n최적화 코드를 반복하여 수행하는 train_loop와 테스트 뎅터로 모델의 성늘을 측정하는 test_loop를 정의하였습니다.\n'

In [106]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        #예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred,y)
        
        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   #https://anweh.tistory.com/22
        
        if batch % 100 == 0:
            loss,current = loss.item(), batch * len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")
            
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [107]:
'''
손실 함수와 옵티마이저를 초기화하고 train_loop와 test_loop에 전달합니다. 모델의 성능 향상을 알아보기 위해 자유롭게 에폭수를 증가시켜 볼 수 있습니다.
'''

'\n손실 함수와 옵티마이저를 초기화하고 train_loop와 test_loop에 전달합니다. 모델의 성능 향상을 알아보기 위해 자유롭게 에폭수를 증가시켜 볼 수 있습니다.\n'

In [108]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n ----------------------------")
    train_loop(train_dataloader, model, loss_fn,optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 ----------------------------
loss:1.033977 [    0/60000]
loss:1.059021 [ 6400/60000]
loss:0.851515 [12800/60000]
loss:1.014749 [19200/60000]
loss:0.900066 [25600/60000]
loss:0.922235 [32000/60000]
loss:0.974624 [38400/60000]
loss:0.910370 [44800/60000]
loss:0.949565 [51200/60000]
loss:0.894357 [57600/60000]
Test Error: 
 Accuracy: 67.1%, Avg loss: 0.906123

Epoch 2
 ----------------------------
loss:0.950442 [    0/60000]
loss:0.995430 [ 6400/60000]
loss:0.773522 [12800/60000]
loss:0.954057 [19200/60000]
loss:0.845709 [25600/60000]
loss:0.855196 [32000/60000]
loss:0.922437 [38400/60000]
loss:0.863535 [44800/60000]
loss:0.893513 [51200/60000]
loss:0.847116 [57600/60000]
Test Error: 
 Accuracy: 68.4%, Avg loss: 0.856020

Epoch 3
 ----------------------------
loss:0.884325 [    0/60000]
loss:0.946149 [ 6400/60000]
loss:0.714151 [12800/60000]
loss:0.907585 [19200/60000]
loss:0.806220 [25600/60000]
loss:0.803973 [32000/60000]
loss:0.881998 [38400/60000]
loss:0.830615 [44800/60000]
